# Ensemble Learning - Bagging 

### 수도 코드는 강의 자료에 있는 것을 채택 

### 직접 구현한 모델에 bagging 을 적용하는 것이 쉽지가 않음. 추후 library를 사용하면 bagging 자체는 쉽게 할 수 있을 것으로 보임. 


##### Algoorightm 1 : Bagging 
Input : Required ensemble size T 
Input : Training set S = {(x1, y1), (x2, y2), ..., (xN, yN)}
for t = 1 to T do 
- build a dataset $S_t$, by sampling N items, randomly with replacement from S. 
- Train a model hy using $S_t$, and add it to the ensemble 
- end for 

for a new testing point (x', y'), 
If model outputs are continuous, combine them by averaging 
If model outputs are class labls, combine them by voting 


**구현해야 하는 것**
- 복원 랜덤 추출 
- OOB를 통한 학습 성능 검증 => 이번에는 미적용. 
- Aggregating 방법 

**필요한 것**
- X : 원본 데이터 셋 
- 적용할 모델 : ex)- SVM, KLDA
- T : 원하는 앙상블 사이즈 
- N : 랜덤 추출할 개수 
- Agg_type : Aggregating 방법 

**함수의 형태**
- def bagging(X, x_new, model, T, N, kernel, Agg_type) : 

- def aggregating(result, agg_type) : 

In [8]:
# 적용할 모델로 KLDA 채택 
# 오류 발생 : X1, X2 의 크기가 맞지 않을 경우, make_n 부분에서 오류가 발생함. 
# Inv_matrix를 구하는 과정에서 Singular Matrix여서 안나오는 경우는 어쩔 수 없음.. 


class KLDA() : 
    
    def __init__(self,X1, X2, kernel) : 
        self.X1 = X1
        self.n1 = np.shape(X1)[0]
        self.m1 = np.shape(X1)[1]
        
        self.X2 = X2 
        self.n2 = np.shape(X2)[0]
        self.m2 = np.shape(X2)[1]
        
        self.kernel = kernel 
        self.k_matrix1, self.k_matrix2 = self.kernel_matrix()
        self.alpha = self.make_a()
        self.project_matrix1, self.project_matrix2  = self.projection_matrix()
        
    def kernel_fn(self, x_i, x_j, kernel = None, coef = 1.0, degree = 3, gamma = 0.1) : 
        if kernel == "poly" : 
            return (np.dot(x_i, x_j) + coef)** degree 
        
        elif kernel == "RBF" : 
            return np.exp(- np.sum(np.square(x_i - x_j)) * gamma)
        
        elif kernel == "sig" : 
            return np.tanh(np.dot(x_i, x_j) + coef) 
        
        else : 
            return np.dot(x_i, x_j) 
        
    def kernel_matrix (self) : 
        k_matrix1 = np.zeros((self.n1, self.n1))
        k_matrix2 = np.zeros((self.n2, self.n2))
        for i in range(self.n1) : 
            for j in range(self.n1) :
                k_matrix1[i,j] = self.kernel_fn(self.X1[i], self.X1[j], self.kernel)
        
        for i in range(self.n2) : 
            for j in range(self.n2) : 
                k_matrix2[i,j] = self.kernel_fn(self.X2[i], self.X2[j], self.kernel)
        
        return k_matrix1, k_matrix2 
    
                
    def make_n(self) : # N, inv_N 제작
        n_matrix = np.zeros((self.n1, self.n1))
        n_matrix = n_matrix + self.k_matrix1 @ (np.identity(self.n1) - (np.ones((self.n1, self.n1))/self.n1)) @ self.k_matrix1.T 
        n_matrix = n_matrix + self.k_matrix2 @ (np.identity(self.n2) - (np.ones((self.n2, self.n2))/self.n2)) @ self.k_matrix2.T
        return n_matrix
        
    
    def mu(self) : 
        mu_1 = self.k_matrix1.sum(axis=1).T
        mu_2 = self.k_matrix2.sum(axis=1).T
        
        return mu_1, mu_2 
    
    def make_a(self) : # 라그랑주 상수 alpha 값 계산 
        n_matrix = self.make_n() 
        n_matrix_inv = np.linalg.inv(n_matrix)
        mu_1, mu_2 = self.mu()
        return n_matrix_inv @ (mu_2 - mu_1) 
    
    def projection(self,x, X) : 
        y = np.sum([self.alpha[i]* self.kernel_fn(X[i], x, kernel = self.kernel) for i in range(len(X))])
        return y
        
    def projection_matrix(self) : 
        matrix_1 = [] 
        matrix_2 = [] 
        for i in range(len(X1)) : 
            matrix_1.append(self.projection(X1[i], X1))
        
        for j in range(len(X2)) : 
            matrix_2.append(self.projection(X2[j], X2))
        
        return matrix_1, matrix_2
    
    def classify(self,x) : 
        range_1 = [min(self.project_matrix1), max(self.project_matrix1)]
        range_2 = [min(self.project_matrix2), max(self.project_matrix2)]
        
        p_x = [self.projection(x, X1), self.projection(x, X2)]
        if min(self.project_matrix1) <= p_x[0] <= max(self.project_matrix1) : 
            return 0
        
        if min(self.project_matrix2) <= p_x[1] <= max(self.project_matrix1) : 
            return 1
        
        else : 
            print("Wait Second :). It's looks like abnormal")
        

In [30]:
import numpy as np
import pandas as pd
import random as rand

from sklearn.datasets import load_iris
X = load_iris()['data']
y = load_iris()["target"].reshape(-1,1)
S = np.concatenate((X,y), axis=1)

import matplotlib.pyplot as plt
import scipy as sc
from scipy.stats import norm
from sys import maxsize

In [65]:
# 이진 분류 모델 
def bagging(X, x_new, model, T, N, kernel, agg_type) : 
    sample_list = []
    ensemble_list = [] 
    X1 = np.array([X[i][:-1] for i in range(len(X)) if X[i][-1] == 0]) 
    X2 = np.array([X[i][:-1] for i in range(len(X)) if X[i][-1] == 1]) 
    
    for i in range(T) : 
        np.random.seed(i)
        sample_index = np.random.choice(np.shape(X1)[0], size = N, replace = True)
        sample_1 = X1[sample_index]
        sample_2 = X2[sample_index]
        ensemble_list.append(model(sample_1,sample_2,kernel)) 
    
    result = [] 
    for i in range(T) : 
        result.append(ensemble_list[i].classify(x_new)) 
    
    return aggregation(result, agg_type)


In [40]:
def aggregation(result, agg_type) : 
    
    if agg_type == "Majority" : 
        agg_result = argmax([result.count(0), result.count(1)]) 
        return agg_result
    
    elif agg_type == "Accuracy Weighted" : 
        return 0
        # 모델별 정확도가 구현되어 있지 않아 여기선 생략 
    
    elif agg_type == "Prediction Weighted" : 
        return 0 
        # 모델별 예상 확률이 구현되어 있지 않아 여기선 생략 
    
        
        

In [66]:
x = [5.1, 3.5, 1.4, 0.2]
test = bagging(S,x, KLDA, 10, 30, "", "Majority")



LinAlgError: Singular matrix

In [50]:
np.random.choice(X[9], 10, True)

array([4.9, 0.1, 3.1, 4.9, 0.1, 0.1, 0.1, 0.1, 3.1, 0.1])

In [41]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3